In [1]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import pandas as pd
%jsroot on

Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0

In [3]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,95,0,20,20,40,40,60,60,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.1,1.4,1.9,2.5,3.5,5.0]
N_pt = len(pt_boarders)-1
file_path="/home/yoren/bnl/PHENIX/ee/AuAu/"
file_name="my-10M_piminus_embed_helios_v4.root"
part_name = ["e^{+}","e^{-}"]

In [4]:
root.gSystem.Load("../AnaTrain/Run14AuAuLeptonComby/MyEvent_C.so")

0

In [5]:
root.gROOT.ProcessLine(
f"TFile *input = new TFile(\"{file_path+file_name}\", \"READ\");\
TTree *T = (TTree *)input->Get(\"tree\");\
MyDileptonAnalysis::MyEvent *myevent = 0;\
TBranch *br = T->GetBranch(\"MyEvent\");\
br->SetAddress(&myevent);\
MyDileptonAnalysis::MyEventContainer *event_container = new MyDileptonAnalysis::MyEventContainer();\
event_container->InitEvent();\
event_container->GetHistsFromFile(\"../../ee_QA/AnaTrain/Run14AuAuLeptonComby/field_map.root\");\
"
); 
from ROOT import myevent, br, event_container

File opened at ../../ee_QA/AnaTrain/Run14AuAuLeptonComby/field_map.root


mcid: 2 - positron, 3 - electron, 8 - pion+, 9 - pion minus

In [6]:
geant3dict = {
  2: "positron",
  3: "electron",
  5: "+muon",
  6: "-muon",
  8: "+pion",
  9: "-pion",
  11: "+kaon",
  12: "-kaon",
  14: "proton",
  15: "antiproton"
}

In [7]:
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
for ievent in range(N_real_ev):
    if good_event_counter>60: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.8 and myevent.GetEntry(ipart).GetN0()>0:
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    print(f"Nevent: {ievent}, centrality: {myevent.GetCentrality()}, x: {myevent.GetPreciseX()}, Ntracks: {myevent.GetNtrack()}") 
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        print("pt: ", round(myevent.GetEntry(ipart).GetPtPrime(),3),", e/p: ",\
              round(myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),3), ", n0: ",\
              myevent.GetEntry(ipart).GetN0(),", disp: ", round(myevent.GetEntry(ipart).GetDisp(),2), ", chi2/npe0: ",\
              round(myevent.GetEntry(ipart).GetChi2()/(myevent.GetEntry(ipart).GetNpe0()+1e-4),2), ", prob: ",\
              round(myevent.GetEntry(ipart).GetProb(),3), ", id: ",  geant3dict[myevent.GetEntry(ipart).GetMcId()])
    for ipart in range(myevent.GetNgentrack()):
        print(myevent.GetGenTrack(ipart).GetPt())

9978832
Nevent: 51, centrality: 76.0, x: 0.34045299887657166, Ntracks: 1
pt:  1.625 , e/p:  0.978 , n0:  1 , disp:  8.06 , chi2/npe0:  30.87 , prob:  0.0 , id:  -pion
1.6208878755569458
Nevent: 502, centrality: 57.0, x: 0.3396129906177521, Ntracks: 1
pt:  0.195 , e/p:  0.867 , n0:  1 , disp:  4.7 , chi2/npe0:  0.25 , prob:  0.809 , id:  -muon
0.5968132019042969
Nevent: 1696, centrality: 19.0, x: 0.32136300206184387, Ntracks: 1
pt:  0.793 , e/p:  0.837 , n0:  1 , disp:  6.18 , chi2/npe0:  0.0 , prob:  0.892 , id:  -pion
0.8034681081771851
Nevent: 4062, centrality: 15.0, x: 0.3496379852294922, Ntracks: 1
pt:  0.431 , e/p:  0.846 , n0:  1 , disp:  5.8 , chi2/npe0:  166115.23 , prob:  0.529 , id:  -pion
0.4389713406562805
Nevent: 4451, centrality: 5.0, x: 0.338811993598938, Ntracks: 1
pt:  0.727 , e/p:  0.954 , n0:  3 , disp:  2.49 , chi2/npe0:  3.1 , prob:  0.187 , id:  -pion
0.7070632576942444
Nevent: 5101, centrality: 40.0, x: 0.328918993473053, Ntracks: 1
pt:  0.253 , e/p:  0.952 , n0:

In [8]:
data_array = []
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
for ievent in range(N_real_ev):
    if good_event_counter>60000000: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==9 and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.6 or myevent.GetEntry(ipart).GetN0()>=0):
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    #print(f"Nevent: {ievent}, centrality: {myevent.GetCentrality()}, x: {myevent.GetPreciseX()}, Ntracks: {myevent.GetNtrack()}") 
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        #print("pt: ", round(myevent.GetEntry(ipart).GetPtPrime(),3),", e/p: ",\
        #      round(myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),3), ", n0: ",\
        #      myevent.GetEntry(ipart).GetN0(),", disp: ", round(myevent.GetEntry(ipart).GetDisp(),2), ", chi2/npe0: ",\
        #      round(myevent.GetEntry(ipart).GetChi2()/(myevent.GetEntry(ipart).GetNpe0()+1e-4),2), ", prob: ",\
        #      round(myevent.GetEntry(ipart).GetProb(),3), ", id: ",  geant3dict[myevent.GetEntry(ipart).GetMcId()])
        data_array.append([int(myevent.GetCentrality()),myevent.GetPreciseZ(),myevent.GetEntry(ipart).GetPtPrime(),\
                           myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),myevent.GetEntry(ipart).GetN0(),\
                           myevent.GetEntry(ipart).GetDisp(),myevent.GetEntry(ipart).GetChi2(),myevent.GetEntry(ipart).GetNpe0(),\
                           myevent.GetEntry(ipart).GetProb(),myevent.GetEntry(ipart).GetPhi0Prime()   ])
    #for ipart in range(myevent.GetNgentrack()):
    #    print(myevent.GetGenTrack(ipart).GetPt())


data = pd.DataFrame(data_array, columns=('centrality','zvertex','pt', 'e/p', 'n0', 'disp', 'chi2', 'npe0', 'prob', 'phi0'))

9978832


In [9]:
data.head()

,centrality,zvertex,pt,e/p,n0,disp,chi2,npe0,prob,phi0
0,80,9.77001,0.507170,0.713720,-9999,-9999.000000,-9999.000000,-9999,0.589843,1.079204
1,71,7.96811,0.642693,0.246567,1,4.419232,24.009247,1,0.905189,3.881259
2,65,4.87799,0.419040,1.562436,-9999,-9999.000000,-9999.000000,-9999,0.010589,0.213936
3,76,-8.97881,1.625024,0.977770,1,8.062794,30.876894,1,0.000002,3.524181
4,84,6.56776,0.379698,0.603102,-9999,-9999.000000,-9999.000000,-9999,0.820395,1.530959


In [10]:
print(data.size, data.shape, good_event_counter)

7053880 (705388, 10) 697979


In [15]:
filename = 'pionminus'
filepath = '/home/yoren/bnl/PHENIX/ee/ML/'+filename
compression_options = dict(method='zip', archive_name=f'{filename}.csv')

In [16]:
data.to_csv(f'{filepath}.zip', compresfilenamen=compression_options, index=False)